# The notebook is to demonstrate Indicating problem

Generate $|z'\rangle$ from quantum part

2-regular graph with 4 nodes(a ring with $Z_4=Z_0$)
    \begin{eqnarray}
        H_c=-\sum_{i=0}^3\frac{Z_iZ_{i+1}}{2},\quad  H_b =\sum_{i=0}^3 X_i .
    \end{eqnarray}

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import expm


# Tensor product function
def tensor_product(*args):
    result = np.array([1])
    for arg in args:
        result = np.kron(result, arg)
    return result

# Unitary evolution operators
def unitary(H, theta):
    return expm(-1j*H*theta)


# Full circuit
def apply_circuit(gamma, beta, p, initial_state, H_c,H_b):
    state = initial_state
    # Define the Hamiltonians H_c and H_b
    for _ in range(p):
        state = unitary(H_c, gamma) @ state
        state = unitary(H_b, beta) @ state
    return state


# Cost function
def cost_function(params, p, initial_state, H_c, H_b):
    gamma, beta = params
  
    final_state = apply_circuit(gamma, beta, p, initial_state, H_c, H_b)
    expectation_value = np.real(np.vdot(final_state, H_c @ final_state))
    
    # Increment the counter and print every 10 iterations
    global iteration_counter
    iteration_counter += 1
    if iteration_counter % 10 == 0:
        print(f'Iteration: {iteration_counter}, gamma: {gamma:.4f}, beta: {beta:.4f}, cost: {expectation_value:.4f}')

    
    
    return expectation_value


def find_ground_energy(hamiltonian):
    """
    Find the ground energy of a Hamiltonian.
    
    Parameters:
    hamiltonian (np.ndarray): The Hamiltonian matrix.
    
    Returns:
    float: The ground energy (lowest eigenvalue) of the Hamiltonian.
    """
    # Calculate the eigenvalues of the Hamiltonian
    eigenvalues = np.linalg.eigvalsh(hamiltonian)
    
    # Return the smallest eigenvalue
    return np.min(eigenvalues)

Pre-train with an initial PQC, circuit formulates like this: $[e ^{-i H_b \beta}e^{-iH_c \gamma}]$

In [2]:
# Define Pauli matrices
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Initial state |+>^4 = (|0> + |1>)/sqrt(2) ⊗ 4
initial_state = tensor_product(np.array([1, 1]) / np.sqrt(2), 
                               np.array([1, 1]) / np.sqrt(2),
                               np.array([1, 1]) / np.sqrt(2),
                               np.array([1, 1]) / np.sqrt(2))
# Define the Hamiltonians H_c and H_b
H_c = -(tensor_product(Z, Z, I, I)+tensor_product(I, Z, Z, I)+tensor_product(I, I, Z, Z)+tensor_product(Z, I, I, Z))
H_b = tensor_product(X, I, I, I)+tensor_product(I, X, I, I)+tensor_product(I, I, X, I)+tensor_product(I, I, I, X)  

# Global iteration counter
iteration_counter = 0

# Optimize using scipy
p = 1  # Number of layers
initial_params = np.random.rand(2)  # Initial guess for gamma and beta

result = minimize(cost_function, initial_params, args=(p,initial_state, H_c, H_b,), method='COBYLA')
optimal_gamma, optimal_beta = result.x

print(f'Optimal gamma: {optimal_gamma}, Optimal beta: {optimal_beta}')


temp_eig3 = find_ground_energy(H_c)
print(temp_eig3)

Iteration: 10, gamma: 1.1001, beta: 0.2836, cost: -1.7250
Iteration: 20, gamma: 1.1802, beta: 0.3920, cost: -1.9999
Iteration: 30, gamma: 1.1782, beta: 0.3927, cost: -2.0000
Optimal gamma: 1.1781634809831525, Optimal beta: 0.3927193541426262
-4.0


Then, using quantum gradient algorithm, we can obtain an updated $|z'\rangle$

In [3]:
def fidelity(state1, state2):

    state1 = np.asarray(state1)
    state2 = np.asarray(state2)
    
#     print(np.vdot(state1, state1))
    # Normalize the states
    state1 = state1 / np.linalg.norm(state1)
    state2 = state2 / np.linalg.norm(state2)
    
    # Calculate the fidelity
    fidelity_value = np.abs(np.vdot(state1, state2))**2
    
    return fidelity_value


def gradient(hamiltonian, xi):
    """
    Compute the gradient of the Hamiltonian with respect to a parameter.

    Parameters:
    hamiltonian (numpy.ndarray): The Hamiltonian matrix.
    xi (float): The scalar value representing the parameter with respect to which 
                the gradient is being computed.

    Returns:
    numpy.ndarray: The gradient matrix, which is the Hamiltonian scaled by the parameter xi.
    """
    
    # Compute the gradient by scaling the Hamiltonian with the parameter xi
    gradient = xi * hamiltonian
    
    # Return the resulting gradient matrix
    return gradient

def PQC_RL(state_in, state_out, env):
    """
    Compute parameterized quantum circuit that tranform state_in to state_out.

    Parameters:
    state_in (numpy.ndarray): the initial state.
    state_out (numpy.ndarray): the end state.
    max_depth (int): maximum depth of the circuit.

    Returns:
    ...: the learned policy.
    """


    # Parameters
    gamma = 0.99
    n_epochs = 4
    clip_range = 0.2
    learning_rate = 0.0001
    policy_kwargs = dict(optimizer_class=optim.Adam)

    # Agent
    ppo_model = PPO("MlpPolicy",
                    env,
                    gamma=gamma,
                    n_epochs=n_epochs,
                    clip_range=clip_range,
                    learning_rate=learning_rate,
                    policy_kwargs=policy_kwargs,
                    tensorboard_log='logs/')
    
    ppo_model.learn(total_timesteps=1000)
    
    # simulate to get fidelity
    state = env.reset()
    print(state)
    done = False
    while not done:
        action = ppo_model.predict(state)
        state, reward, done, info = env.step(action[0])
        #展示当前的线路 和 state
        env.render()
        # print(state)
        print(info['fidelity'])
    return info,ppo_model



In [4]:
import gym
import numpy as np
import torch.optim as optim
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from qulacs import QuantumState
import qas_gym

xi=0.1

state_in=apply_circuit(optimal_gamma, optimal_beta, p, initial_state, H_c,H_b)

state_out=state_in-gradient(H_c, xi)@state_in

state_out = state_out/np.linalg.norm(state_out)

# # print(state_out)

# n = 4
# state = QuantumState(n)
# state.set_Haar_random_state(seed=1)
# state_in = state.get_vector()
# state.set_Haar_random_state(seed=2)
# state_out = state.get_vector()
print(fidelity(state_out, state_in))


# fidelity(state_out, state_out_pqc)

# expectation_value = np.real(np.vdot(state_out, H_c @ state_out))
# print(expectation_value)


env_name = 'BasicFourQubit-v0' # 四比特环境-无噪声
fidelity_threshold = 0.98
reward_penalty = 0.01
# max_timesteps = 5
max_depth = 5
env = gym.make(env_name, target = state_out,
        fidelity_threshold=fidelity_threshold,
        reward_penalty=reward_penalty,
        max_timesteps=max_depth,
        initial = state_in)


info,ppo_model = PQC_RL(state_in, state_out, env)


0.9664453371357377


C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\

number of gates:1,  number of param gates:1
Fidelity: 0.9686795407044685
number of gates:2,  number of param gates:2
Fidelity: 0.9751367196298405
number of gates:3,  number of param gates:3
Fidelity: 0.9751367202454176
number of gates:4,  number of param gates:4
Fidelity: 0.9751367190707421
number of gates:5,  number of param gates:5
Fidelity: 0.9764650164231066
number of gates:1,  number of param gates:1
Fidelity: 0.9686795399888347
number of gates:2,  number of param gates:2
Fidelity: 0.9751367196817253
number of gates:3,  number of param gates:3
Fidelity: 0.9764650214651622
number of gates:4,  number of param gates:4
Fidelity: 0.3153885753009109
number of gates:5,  number of param gates:5
Fidelity: 0.32080344158141977
number of gates:1,  number of param gates:1
Fidelity: 0.973456351677155
number of gates:2,  number of param gates:2
Fidelity: 0.9734563514721655
number of gates:3,  number of param gates:3
Fidelity: 0.9760777072058555
number of gates:4,  number of param gates:4
Fidelit

C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


In [5]:
# Visualize the final circuit
env.render()


          ┌────┐
0: ───I────XX────────────────YY───
           │                 │
1: ───I────┼─XX────XX────────┼────
           │ │     │         │
2: ───I────XX┼─────XX───ZZ───┼────
             │          │    │
3: ───I──────XX─────────ZZ───YY───
          └────┘
